
Unsupervised Learning Trading Strategy

    Download/Load SP500 stocks prices data.
    Calculate different features and indicators on each stock.
    Aggregate on monthly level and filter top 150 most liquid stocks.
    Calculate Monthly Returns for different time-horizons.
    Download Fama-French Factors and Calculate Rolling Factor Betas.
    For each month fit a K-Means Clustering Algorithm to group similar assets based on their features.
    For each month select assets based on the cluster and form a portfolio based on Efficient Frontier max sharpe ratio optimization.
    Visualize Portfolio returns and compare to SP500 returns.




All Packages Needed:

    pandas, numpy, matplotlib, statsmodels, pandas_datareader, datetime, yfinance, sklearn, PyPortfolioOpt



In [6]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

In [8]:
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2024-09-23'

start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date).stack()

df.index.names = ['date', 'ticker']

df.columns = df.columns.str.lower()

df

[*********************100%***********************]  503 of 503 completed


Price                              adj close       close        high  \
date                      ticker                                       
2016-09-26 00:00:00+00:00 A        43.734444   46.570000   46.770000   
                          AAPL     26.029974   28.219999   28.347500   
                          ABBV     45.387066   64.070000   64.879997   
                          ABT      36.004772   41.680000   42.020000   
                          ACGL     26.486668   26.486668   26.653334   
...                                      ...         ...         ...   
2024-09-20 00:00:00+00:00 XYL     134.809998  134.809998  136.309998   
                          YUM     129.710007  129.710007  130.910004   
                          ZBH     106.419998  106.419998  106.739998   
                          ZBRA    366.760010  366.760010  367.559998   
                          ZTS     192.639999  192.639999  194.550003   

Price                                    low        open       volume  
date                      ticker                                       
2016-09-26 00:00:00+00:00 A        46.160000   46.459999    2639100.0  
                          AAPL     27.887501   27.910000  119477600.0  
                          ABBV     64.019997   64.519997    5567700.0  
                          ABT      41.610001   42.009998    6378200.0  
                          ACGL     26.463333   26.653334    1023600.0  
...                                      ...         ...          ...  
2024-09-20 00:00:00+00:00 XYL     133.770004  135.820007    2267000.0  
                          YUM     128.720001  130.759995    3272700.0  
                          ZBH     105.010002  105.570000    3920000.0  
                          ZBRA    360.019989  362.429993     596300.0  
                          ZTS     191.070007  192.940002    3520700.0  

[989692 rows x 6 columns]

In [9]:
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
                                                          
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
                                                          
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
                        low=stock_data['low'],
                        close=stock_data['close'],
                        length=14)
    return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

df['dollar_volume'] = (df['adj close']*df['volume'])/1e6

df

Price                              adj close       close        high  \
date                      ticker                                       
2016-09-26 00:00:00+00:00 A        43.734444   46.570000   46.770000   
                          AAPL     26.029974   28.219999   28.347500   
                          ABBV     45.387066   64.070000   64.879997   
                          ABT      36.004772   41.680000   42.020000   
                          ACGL     26.486668   26.486668   26.653334   
...                                      ...         ...         ...   
2024-09-20 00:00:00+00:00 XYL     134.809998  134.809998  136.309998   
                          YUM     129.710007  129.710007  130.910004   
                          ZBH     106.419998  106.419998  106.739998   
                          ZBRA    366.760010  366.760010  367.559998   
                          ZTS     192.639999  192.639999  194.550003   

Price                                    low        open       volume  \
date                      ticker                                        
2016-09-26 00:00:00+00:00 A        46.160000   46.459999    2639100.0   
                          AAPL     27.887501   27.910000  119477600.0   
                          ABBV     64.019997   64.519997    5567700.0   
                          ABT      41.610001   42.009998    6378200.0   
                          ACGL     26.463333   26.653334    1023600.0   
...                                      ...         ...          ...   
2024-09-20 00:00:00+00:00 XYL     133.770004  135.820007    2267000.0   
                          YUM     128.720001  130.759995    3272700.0   
                          ZBH     105.010002  105.570000    3920000.0   
                          ZBRA    360.019989  362.429993     596300.0   
                          ZTS     191.070007  192.940002    3520700.0   

Price                             garman_klass_vol        rsi    bb_low  \
date                      ticker                                          
2016-09-26 00:00:00+00:00 A              -0.001326        NaN       NaN   
                          AAPL           -0.001745        NaN       NaN   
                          ABBV           -0.047706        NaN       NaN   
                          ABT            -0.009144        NaN       NaN   
                          ACGL            0.000010        NaN       NaN   
...                                            ...        ...       ...   
2024-09-20 00:00:00+00:00 XYL             0.000155  53.435794  4.847310   
                          YUM             0.000117  40.091876  4.878913   
                          ZBH             0.000109  43.013301  4.620909   
                          ZBRA            0.000160  65.622235  5.769405   
                          ZTS             0.000162  60.241227  5.191393   

Price                               bb_mid   bb_high       atr      macd  \
date                      ticker                                           
2016-09-26 00:00:00+00:00 A            NaN       NaN       NaN       NaN   
                          AAPL         NaN       NaN       NaN       NaN   
                          ABBV         NaN       NaN       NaN       NaN   
                          ABT          NaN       NaN       NaN       NaN   
                          ACGL         NaN       NaN       NaN       NaN   
...                                    ...       ...       ...       ...   
2024-09-20 00:00:00+00:00 XYL     4.894404  4.941498  1.038409 -0.081219   
                          YUM     4.902032  4.925151  0.403419 -0.750091   
                          ZBH     4.703192  4.785475 -0.617732 -0.543000   
                          ZBRA    5.837354  5.905304  0.276729  0.691584   
                          ZTS     5.243462  5.295530  0.648675  1.099930   

Price                             dollar_volume  
date                      ticker                 
2016-09-26 00:00:00+00:00 A          115.419570 


3. Aggregate to monthly level and filter top 150 most liquid stocks for each month.

    To reduce training time and experiment with features and strategies, we convert the business-daily data to month-end frequency.



In [10]:
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open',
                                                          'high', 'low', 'close']]

data = (pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume'),
                   df.unstack()[last_cols].resample('M').last().stack('ticker')],
                  axis=1)).dropna()

data


dollar_volume   adj close       atr  \
date                      ticker                                        
2016-10-31 00:00:00+00:00 A           75.765042   41.018742 -1.372955   
                          AAPL      3494.595217   26.182171 -1.302691   
                          ABBV       274.543306   39.878784 -1.002986   
                          ABT        317.927975   34.112484 -1.252137   
                          ACGL        29.912385   25.990000 -1.086342   
...                                         ...         ...       ...   
2024-09-30 00:00:00+00:00 XYL        173.726560  134.809998  1.038409   
                          YUM        277.875901  129.710007  0.403419   
                          ZBH        213.235628  106.419998 -0.617732   
                          ZBRA       133.237764  366.760010  0.276729   
                          ZTS        388.163838  192.639999  0.648675   

                                   bb_high    bb_low    bb_mid  \
date                      ticker                                 
2016-10-31 00:00:00+00:00 A       3.839099  3.718486  3.778792   
                          AAPL    3.350880  3.293123  3.322002   
                          ABBV    3.852823  3.744517  3.798670   
                          ABT     3.650426  3.549492  3.599959   
                          ACGL    3.322517  3.278161  3.300339   
...                                    ...       ...       ...   
2024-09-30 00:00:00+00:00 XYL     4.941498  4.847310  4.894404   
                          YUM     4.925151  4.878913  4.902032   
                          ZBH     4.785475  4.620909  4.703192   
                          ZBRA    5.905304  5.769405  5.837354   
                          ZTS     5.295530  5.191393  5.243462   

                                  garman_klass_vol      macd        rsi  
date                      ticker                                         
2016-10-31 00:00:00+00:00 A              -0.001089 -0.809474  32.795973  
                          AAPL           -0.002541 -0.275727  43.456241  
                          ABBV           -0.049190 -0.864523  22.957314  
                          ABT            -0.008074 -0.839187  36.011892  
                          ACGL            0.000021 -0.507689  46.128721  
...                                            ...       ...        ...  
2024-09-30 00:00:00+00:00 XYL             0.000155 -0.081219  53.435794  
                          YUM             0.000117 -0.750091  40.091876  
                          ZBH             0.000109 -0.543000  43.013301  
                          ZBRA            0.000160  0.691584  65.622235  
                          ZTS             0.000162  1.099930  60.241227  

[47253 rows x 9 columns]

Calculate 5-year rolling average of dollar volume for each stocks before filtering.

In [11]:
data['dollar_volume'] = (data.loc[:, 'dollar_volume'].unstack('ticker').rolling(5*12, min_periods=12).mean().stack())

data['dollar_vol_rank'] = (data.groupby('date')['dollar_volume'].rank(ascending=False))

data = data[data['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)

data

adj close       atr   bb_high    bb_low  \
date                      ticker                                             
2017-09-30 00:00:00+00:00 AAPL     36.166759 -1.161410  3.684698  3.589136   
                          ABBV     65.390602 -0.479591  4.254232  4.050517   
                          ABT      47.232571 -1.192285  3.877438  3.825775   
                          ACN     121.064735 -1.129961  4.841457  4.770444   
                          ADBE    149.179993 -1.408016  5.085897  4.977719   
...                                      ...       ...       ...       ...   
2024-09-30 00:00:00+00:00 VZ       44.330002 -0.095145  3.838447  3.718181   
                          WFC      56.290001  1.429790  4.102454  3.966989   
                          WMT      79.059998  1.902732  4.404047  4.329609   
                          XOM     115.269997  0.874633  4.794201  4.705064   
                          ZTS     192.639999  0.648675  5.295530  5.191393   

                                    bb_mid  garman_klass_vol      macd  \
date                      ticker                                         
2017-09-30 00:00:00+00:00 AAPL    3.636917         -0.001175 -0.458862   
                          ABBV    4.152375         -0.035700  1.019895   
                          ABT     3.851606         -0.006208  0.324929   
                          ACN     4.805951         -0.005643  0.160990   
                          ADBE    5.031808          0.000055 -0.295938   
...                                    ...               ...       ...   
2024-09-30 00:00:00+00:00 VZ      3.778314          0.000113  1.882308   
                          WFC     4.034722          0.000212 -0.468529   
                          WMT     4.366828          0.000138  2.901494   
                          XOM     4.749633          0.000059 -0.484281   
                          ZTS     5.243462          0.000162  1.099930   

                                        rsi  
date                      ticker             
2017-09-30 00:00:00+00:00 AAPL    45.768751  
                          ABBV    70.568661  
                          ABT     63.665092  
                          ACN     56.250037  
                          ADBE    47.932457  
...                                     ...  
2024-09-30 00:00:00+00:00 VZ      63.040916  
                          WFC     52.139004  
                          WMT     65.861520  
                          XOM     51.315440  
                          ZTS     60.241227  

[12665 rows x 8 columns]


4. Calculate Monthly Returns for different time horizons as features.

    To capture time series dynamics that reflect, for example, momentum patterns, we compute historical returns using the method .pct_change(lag), that is, returns over various monthly periods as identified by lags.



In [12]:
def calculate_returns(df):

    outlier_cutoff = 0.005

    lags = [1, 2, 3, 6, 9, 12]

    for lag in lags:

        df[f'return_{lag}m'] = (df['adj close']
                              .pct_change(lag)
                              .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                     upper=x.quantile(1-outlier_cutoff)))
                              .add(1)
                              .pow(1/lag)
                              .sub(1))
    return df
    
    
data = data.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()

data

adj close       atr   bb_high    bb_low  \
date                      ticker                                             
2018-09-30 00:00:00+00:00 AAPL     53.774536 -0.893283  4.018120  3.958415   
                          ABBV     72.047684 -0.712845  4.312119  4.257714   
                          ABT      66.177368 -1.104782  4.211202  4.073479   
                          ACN     155.388870 -1.089805  5.077972  5.037983   
                          ADBE    269.950012 -0.817711  5.619120  5.555771   
...                                      ...       ...       ...       ...   
2024-09-30 00:00:00+00:00 VRTX    464.920013  1.753261  6.208553  6.139200   
                          VZ       44.330002 -0.095145  3.838447  3.718181   
                          WFC      56.290001  1.429790  4.102454  3.966989   
                          WMT      79.059998  1.902732  4.404047  4.329609   
                          XOM     115.269997  0.874633  4.794201  4.705064   

                                    bb_mid  garman_klass_vol      macd  \
date                      ticker                                         
2018-09-30 00:00:00+00:00 AAPL    3.988268         -0.000718 -0.053480   
                          ABBV    4.284916         -0.027228 -0.480382   
                          ABT     4.142340         -0.003433  1.151956   
                          ACN     5.057977         -0.003063  0.130998   
                          ADBE    5.587446          0.000100  0.114814   
...                                    ...               ...       ...   
2024-09-30 00:00:00+00:00 VRTX    6.173876          0.000125 -0.714526   
                          VZ      3.778314          0.000113  1.882308   
                          WFC     4.034722          0.000212 -0.468529   
                          WMT     4.366828          0.000138  2.901494   
                          XOM     4.749633          0.000059 -0.484281   

                                        rsi  return_1m  return_2m  return_3m  \
date                      ticker                                               
2018-09-30 00:00:00+00:00 AAPL    61.186198  -0.008303   0.091080   0.069629   
                          ABBV    49.718909  -0.014586   0.012660   0.010312   
                          ABT     79.127063   0.097547   0.057978   0.065082   
                          ACN     54.490650   0.006683   0.033549   0.013291   
                          ADBE    56.971111   0.024439   0.050370   0.034532   
...                                     ...        ...        ...        ...   
2024-09-30 00:00:00+00:00 VRTX    43.244471  -0.062453  -0.031564  -0.002710   
                          VZ      63.040916   0.061034   0.045958   0.029935   
                          WFC     52.139004  -0.037284  -0.022325  -0.015216   
                          WMT     65.861520   0.023695   0.074751   0.054015   
                          XOM     51.315440  -0.022639  -0.010136   0.003112   

                                  return_6m  return_9m  return_12m  
date                      ticker                                    
2018-09-30 00:00:00+00:00 AAPL     0.051986   0.033843    0.033607  
                          ABBV     0.003293   0.000594    0.008112  
                          ABT      0.035906   0.029900    0.028503  
                          ACN      0.018852   0.012830    0.021018  
                          ADBE     0.037795   0.049180    0.050665  
...                                     ...        ...         ...  
2024-09-30 00:00:00+00:00 VRTX     0.017885   0.014924    0.024496  
                          VZ       0.014647   0.023724    0.028731  
                          WFC     -0.002645   0.017322    0.029560  
                          WMT      0.047657   0.047243    0.033315  
                          XOM      0.001289   0.018804    0.001219  

[10510 rows x 14 columns]


5. Download Fama-French Factors and Calculate Rolling Factor Betas.

    We will introduce the Fama—French data to estimate the exposure of assets to common risk factors using linear regression.

    The five Fama—French factors, namely market risk, size, value, operating profitability, and investment have been shown empirically to explain asset returns and are commonly used to assess the risk/return profile of portfolios. Hence, it is natural to include past factor exposures as financial features in models.

    We can access the historical factor returns using the pandas-datareader and estimate historical exposures using the RollingOLS rolling linear regression.



In [16]:
import pandas_datareader.data as web

# Retrieve factor data and process it
factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3',
                             'famafrench',
                             start='2010')[0].drop('RF', axis=1)

factor_data.index = factor_data.index.to_timestamp()

# Resample and adjust factor data
factor_data = factor_data.resample('M').last().div(100)

factor_data.index.name = 'date'

# Handle the index of data['return_1m'] if it's a MultiIndex
if isinstance(data['return_1m'].index, pd.MultiIndex):
    # If one of the levels is a datetime index, localize it
    if isinstance(data['return_1m'].index.get_level_values(0), pd.DatetimeIndex):
        data['return_1m'].index = data['return_1m'].index.set_levels(
            [data['return_1m'].index.levels[0].tz_localize(None), data['return_1m'].index.levels[1]],
            level=[0, 1]
        )
else:
    # Remove timezone information from index if it's a DatetimeIndex
    data['return_1m'].index = data['return_1m'].index.tz_localize(None)

# Join factor data with returns data and sort by index
factor_data = factor_data.join(data['return_1m']).sort_index()

# Output the final joined DataFrame
factor_data


Mkt-RF     SMB     HML     RMW     CMA  return_1m
date       ticker                                                   
2018-09-30 AAPL    0.0006 -0.0249 -0.0171  0.0063  0.0129  -0.008303
           ABBV    0.0006 -0.0249 -0.0171  0.0063  0.0129  -0.014586
           ABT     0.0006 -0.0249 -0.0171  0.0063  0.0129   0.097547
           ACN     0.0006 -0.0249 -0.0171  0.0063  0.0129   0.006683
           ADBE    0.0006 -0.0249 -0.0171  0.0063  0.0129   0.024439
...                   ...     ...     ...     ...     ...        ...
2024-07-31 VRTX    0.0124  0.0833  0.0570  0.0017  0.0043   0.057604
           VZ      0.0124  0.0833  0.0570  0.0017  0.0043  -0.001379
           WFC     0.0124  0.0833  0.0570  0.0017  0.0043  -0.000842
           WMT     0.0124  0.0833  0.0570  0.0017  0.0043   0.013735
           XOM     0.0124  0.0833  0.0570  0.0017  0.0043   0.030142

[10226 rows x 6 columns]